In [22]:
#读取文件数据，用pkl文件存储比csv更具有速度优势
import pickle
with open('data.pkl','rb') as file:
    train_data= pickle.load(file)
train_data

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,Customer fullname,order_year,order_month,order_week_day,order_hour,order_month_year
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,327.750000,0,2/3/2018 22:56,Standard Class,CallyHolloway,2018,1,2,22,2018-01
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,327.750000,0,1/18/2018 12:27,Standard Class,IreneLuna,2018,1,5,12,2018-01
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,327.750000,0,1/17/2018 12:06,Standard Class,GillianMaldonado,2018,1,5,12,2018-01
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,327.750000,0,1/16/2018 11:45,Standard Class,TanaTate,2018,1,5,11,2018-01
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,327.750000,0,1/15/2018 11:24,Standard Class,OrliHendricks,2018,1,5,11,2018-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,0,45,Fishing,Brooklyn,...,399.980011,0,1/20/2016 3:40,Standard Class,MariaPeterson,2016,1,5,3,2016-01
180515,DEBIT,3,2,-613.770019,395.980011,Late delivery,1,45,Fishing,Bakersfield,...,399.980011,0,1/19/2016 1:34,Second Class,RonaldClark,2016,1,5,1,2016-01
180516,TRANSFER,5,4,141.110001,391.980011,Late delivery,1,45,Fishing,Bristol,...,399.980011,0,1/20/2016 21:00,Standard Class,JohnSmith,2016,1,4,21,2016-01
180517,PAYMENT,3,4,186.229996,387.980011,Advance shipping,0,45,Fishing,Caguas,...,399.980011,0,1/18/2016 20:18,Standard Class,MarySmith,2016,1,4,20,2016-01


In [23]:
#创建字段,构建label
import numpy as np
train_data['fraud'] = np.where(train_data['Order Status'] == 'SUSPECTED_FRAUD',1,0)
#train_data['fraud'].sum()
print(train_data['Delivery Status'].value_counts())
train_data['late_delivery'] = np.where(train_data['Delivery Status'] == 'Late delivery',1,0)
print(train_data['late_delivery'])


Late delivery        98977
Advance shipping     41592
Shipping on time     32196
Shipping canceled     7754
Name: Delivery Status, dtype: int64
0         0
1         1
2         0
3         0
4         0
         ..
180514    0
180515    1
180516    1
180517    0
180518    0
Name: late_delivery, Length: 180519, dtype: int32


In [24]:
train_data.info()#查看数据字段详情

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 61 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Type                           180519 non-null  object        
 1   Days for shipping (real)       180519 non-null  int64         
 2   Days for shipment (scheduled)  180519 non-null  int64         
 3   Benefit per order              180519 non-null  float64       
 4   Sales per customer             180519 non-null  float64       
 5   Delivery Status                180519 non-null  object        
 6   Late_delivery_risk             180519 non-null  int64         
 7   Category Id                    180519 non-null  int64         
 8   Category Name                  180519 non-null  object        
 9   Customer City                  180519 non-null  object        
 10  Customer Country               180519 non-null  object        
 11  

In [25]:
train_data.drop(['Customer Email','Customer Password','Product Description','Product Status','Product Image','Customer Lname','Customer Fname','Order Customer Id','Order Item Cardprod Id','Order Item Id','Sales per customer','Order Item Total','Order Profit Per Order','Product Card Id','Product Category Id','Product Price','Order Zipcode','shipping date (DateOrders)','Latitude','Longitude','Customer Street','order date (DateOrders)','order_month_year','Order Status','Delivery Status','Late_delivery_risk'],axis=1,inplace=True)
#对数据中不必要的字段，相关性高的，标签泄露的（或是与原标签有直接的线性关系的）进行删除列

In [26]:
len(train_data.columns)#查看是否删掉无用字段

35

In [27]:
#选出字段类型为object类型的字段
categorical_cols = train_data.select_dtypes(include='object').columns
categorical_cols

Index(['Type', 'Category Name', 'Customer City', 'Customer Country',
       'Customer Segment', 'Customer State', 'Department Name', 'Market',
       'Order City', 'Order Country', 'Order Region', 'Order State',
       'Product Name', 'Shipping Mode', 'Customer fullname'],
      dtype='object')

In [28]:
train_data =  train_data.dropna(subset=['Customer fullname'])#删除掉null值

In [29]:
#object类型不为数值类型，不能进行运算，这里将所有的object类型利用LabelEncoder进行转换进行计算
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for cat in categorical_cols:
    train_data[cat] = le.fit_transform(train_data[cat])
train_data[categorical_cols]

D:\python\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Type,Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Name,Market,Order City,Order Country,Order Region,Order State,Product Name,Shipping Mode,Customer fullname
0,1,40,66,1,0,36,4,3,331,70,15,475,78,3,1875
1,3,40,66,1,0,36,4,3,391,69,13,841,78,3,5374
2,0,40,452,0,0,5,4,3,391,69,13,841,78,3,4426
3,1,40,285,0,2,5,4,3,3226,8,11,835,78,3,12922
4,2,40,66,1,1,36,4,3,3226,8,11,835,78,3,10632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,0,18,59,0,2,31,3,3,2922,31,7,913,24,3,8421
180515,1,18,26,0,1,5,3,3,1362,77,7,770,24,2,11672
180516,3,18,55,0,1,7,3,3,25,8,11,88,24,3,6296
180517,2,18,66,1,0,36,3,3,25,8,11,88,24,3,9379


In [30]:
#剔除掉所有object类型的列
numeracal_columns= train_data.columns.tolist()
for x in categorical_cols.tolist():
    numeracal_columns.remove(x)
numeracal_columns

['Days for shipping (real)',
 'Days for shipment (scheduled)',
 'Benefit per order',
 'Category Id',
 'Customer Id',
 'Customer Zipcode',
 'Department Id',
 'Order Id',
 'Order Item Discount',
 'Order Item Discount Rate',
 'Order Item Product Price',
 'Order Item Profit Ratio',
 'Order Item Quantity',
 'Sales',
 'order_year',
 'order_month',
 'order_week_day',
 'order_hour',
 'fraud',
 'late_delivery']

In [31]:
#对欺诈类型预测
#切分特征和label
x_fraud = train_data.loc[:, train_data.columns!='fraud']
y_fraud = train_data['fraud']
x_late = train_data.loc[:, train_data.columns!='late_delivery']
y_late = train_data['late_delivery']
x_fraud


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Category Id,Category Name,Customer City,Customer Country,Customer Id,Customer Segment,...,Order Region,Order State,Product Name,Shipping Mode,Customer fullname,order_year,order_month,order_week_day,order_hour,late_delivery
0,1,3,4,91.250000,73,40,66,1,20755,0,...,15,475,78,3,1875,2018,1,2,22,0
1,3,5,4,-249.089996,73,40,66,1,19492,0,...,13,841,78,3,5374,2018,1,5,12,1
2,0,4,4,-247.779999,73,40,452,0,19491,0,...,13,841,78,3,4426,2018,1,5,12,0
3,1,3,4,22.860001,73,40,285,0,19490,2,...,11,835,78,3,12922,2018,1,5,11,0
4,2,2,4,134.210007,73,40,66,1,19489,1,...,11,835,78,3,10632,2018,1,5,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,0,4,4,40.000000,45,18,59,0,1005,2,...,7,913,24,3,8421,2016,1,5,3,0
180515,1,3,2,-613.770019,45,18,26,0,9141,1,...,7,770,24,2,11672,2016,1,5,1,1
180516,3,5,4,141.110001,45,18,55,0,291,1,...,11,88,24,3,6296,2016,1,4,21,1
180517,2,3,4,186.229996,45,18,66,1,2813,0,...,11,88,24,3,9379,2016,1,4,20,0


In [32]:
#切分数据集   下面没啥说的了，按部就班
from sklearn.model_selection import train_test_split
x_fraud_train,x_fraud_test,y_fraud_train,y_fraud_test =train_test_split(x_fraud,y_fraud,test_size=0.2)
x_late_train,x_late_test,y_late_train,y_late_test =train_test_split(x_late,y_late,test_size=0.2)

In [33]:
#数据规范化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_fraud_train = sc.fit_transform(x_fraud_train)
x_fraud_test = sc.transform(x_fraud_test)

x_late_train = sc.fit_transform(x_late_train)
x_late_test = sc.transform(x_late_test)

In [34]:
#定义model和衡量标准
from sklearn.metrics import accuracy_score,recall_score,roc_auc_score,confusion_matrix,f1_score
def model_stats (model,x_train,x_test,y_train,y_test,name='Fraud'):
    model = model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_pred,y_test)
    recall = recall_score(y_pred,y_test,average='micro')
    auc = roc_auc_score(y_pred,y_test)
    confusion = confusion_matrix(y_pred,y_test)
    f1=f1_score(y_pred,y_test)
    print('model USed:',model)
    print('{} accuracy:{}%'.format(name,accuracy*100))
    print('{} recall:{}%'.format(name, recall * 100))
    print('{} auc:{}%'.format(name, auc * 100))
    print('{} confusion:{}%'.format(name, confusion * 100))
    print('{} f1:{}%'.format(name, f1 * 100))
    return auc,recall,f1

In [35]:
#逻辑回归模型
from sklearn.linear_model import LogisticRegression

model_fraud = LogisticRegression()
model_late = LogisticRegression()

model_stats(model_fraud,x_fraud_train,x_fraud_test,y_fraud_train,y_fraud_test,'Fraud')
print('\n')
model_stats(model_late,x_late_train,x_late_test,y_late_train,y_late_test,'late_delivery')

D:\python\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


model USed: LogisticRegression()
Fraud accuracy:97.90876104478852%
Fraud recall:97.90876104478852%
Fraud auc:77.90782883455248%
Fraud confusion:[[3516900   62300]
 [  13200   17900]]%
Fraud f1:32.16531895777179%


model USed: LogisticRegression()
late_delivery accuracy:98.7701853031604%
late_delivery recall:98.7701853031604%
late_delivery auc:98.8980442767795%
late_delivery confusion:[[1595700       0]
 [  44400 1970200]]%
late_delivery f1:98.88576591045975%


(0.9889804427677951, 0.9877018530316041, 0.9888576591045976)

In [43]:
train_data.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Id', 'Customer Segment', 'Customer State',
       'Customer Zipcode', 'Department Id', 'Department Name', 'Market',
       'Order City', 'Order Country', 'Order Id', 'Order Item Discount',
       'Order Item Discount Rate', 'Order Item Product Price',
       'Order Item Profit Ratio', 'Order Item Quantity', 'Sales',
       'Order Region', 'Order State', 'Product Name', 'Shipping Mode',
       'Customer fullname', 'order_year', 'order_month', 'order_week_day',
       'order_hour', 'fraud', 'late_delivery'],
      dtype='object')

In [45]:
x_sales = train_data.loc[:, train_data.columns!='Sales']
y_sales = train_data['Sales']
print(y_sales.head())
print('\n')
x_order = train_data.loc[:, train_data.columns!='Order Item Quantity']
y_order = train_data['Order Item Quantity']
print(x_order.columns)

0    327.75
1    327.75
2    327.75
3    327.75
4    327.75
Name: Sales, dtype: float64


Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Id', 'Customer Segment', 'Customer State',
       'Customer Zipcode', 'Department Id', 'Department Name', 'Market',
       'Order City', 'Order Country', 'Order Id', 'Order Item Discount',
       'Order Item Discount Rate', 'Order Item Product Price',
       'Order Item Profit Ratio', 'Sales', 'Order Region', 'Order State',
       'Product Name', 'Shipping Mode', 'Customer fullname', 'order_year',
       'order_month', 'order_week_day', 'order_hour', 'fraud',
       'late_delivery'],
      dtype='object')


In [48]:
#对销售额和订货数量进行预测
x_sales_train, x_sales_test, y_sales_train, y_sales_test = train_test_split(x_sales,y_sales, test_size=0.2)
x_order_train, x_order_test, y_order_train, y_order_test = train_test_split(x_order,y_order, test_size=0.2)

In [58]:
#定义回归的自定义函数
from sklearn.metrics import mean_absolute_error,mean_squared_error
def model_stats_regression (model,x_train,x_test,y_train,y_test,name='Sales'):
    #使用MAE，MSE作为评价指标
    model = model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print('model USed:',model)
    mse = mean_squared_error(y_pred,y_test)
    mae = mean_absolute_error(y_pred,y_test)
    print('{} mse:{}%'.format(name,mse))
    print('{} mae:{}%'.format(name, mae))
    return mse,mae


In [59]:
#采用线性回归模型
from sklearn.linear_model import LinearRegression
model_sales = LinearRegression()
model_order = LinearRegression()
model_stats_regression(model_sales,x_sales_train,x_sales_test,y_sales_train,y_sales_test,'Sales')


model USed: LinearRegression()
Sales mse:919.3925549873862%
Sales mae:18.23624883177669%


(919.3925549873862, 18.23624883177669)

In [56]:
model_stats_regression(model_order,x_order_train,x_order_test,y_order_train,y_order_test,'order')

model USed: LinearRegression()
order mse:0.28454578182408014%
order mae:0.34798509683853607%


(0.28454578182408014, 0.34798509683853607)